Dataset is [here](https://www.kaggle.com/benroshan/ecommerce-data?select=Order+Details.csv).

# **Questions:**
1. In which order, there was maximum loss and in which order, there was maximum profit?

2. Which category has been the most profitable and the least profitable (maybe have caused a loss as well)?

3. From which state, most orders have been placed? 

4. Which category of product has been sold the most?

5. For which all months, the sales target have been achieved? So, for each month, there's a sales target for each category. Find out for which category and which month, the sales target was achieved and not achieved.

6. Which customer has spent the most amount on the website?

 Installing latest version of jdk

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get url from spark's official site and download it

In [ ]:
# get file url from the spark website
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!ls

sample_data  spark-3.1.2-bin-hadoop2.7.tgz


Unzip the tar file and install findspark to get started with spark

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.2-bin-hadoop2.7'

Create the spark session

In [ ]:
import pyspark
from pyspark.sql import SparkSession


spark=SparkSession.builder.master('local[1]').appName('E-commerce').getOrCreate()

Load the data sets

In [ ]:
order_deets_df = spark.read.csv("/content/Order Details.csv", header = True, inferSchema = True)
orders_df = spark.read.csv("/content/List of Orders.csv", header = True, inferSchema = True)
target_df = spark.read.csv("/content/Sales target.csv", header = True, inferSchema = True)

In [ ]:
order_deets_df.printSchema()
orders_df.printSchema()
target_df.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)

root
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- CustomerName: string (nullable = true)
 |-- State: string (nullable = true)
 |-- City: string (nullable = true)

root
 |-- Month of Order Date: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Target: double (nullable = true)



In [149]:
from pyspark.sql.functions import * 


**1. Max and Min Profit**

In [273]:
mod_order = order_deets_df.groupBy("Order ID").agg(sum("Profit").alias("Profit Per Order")).orderBy(desc("Profit Per Order"))
print("Order with max Profit:",mod_order.first())
print("Order with max loss:",mod_order.tail(1))

Order with max Profit: Row(Order ID='B-25973', Profit Per Order=1970.0)
Order with max loss: [Row(Order ID='B-25798', Profit Per Order=-1836.0)]


**4. Category of Product that is sold the most**

In [ ]:
order_deets_df.select("Category","Quantity").orderBy(desc("Quantity")).show()

+-----------+--------+
|   Category|Quantity|
+-----------+--------+
|  Furniture|      14|
|   Clothing|      14|
|   Clothing|      13|
|   Clothing|      13|
|  Furniture|      13|
|Electronics|      13|
|  Furniture|      12|
|   Clothing|      12|
|   Clothing|      12|
|Electronics|      11|
|Electronics|      11|
|   Clothing|      11|
|Electronics|      11|
|   Clothing|      11|
|Electronics|      11|
|Electronics|      11|
|  Furniture|      11|
|   Clothing|      11|
|   Clothing|      11|
|  Furniture|      10|
+-----------+--------+
only showing top 20 rows



In [97]:
order_deets_df.groupBy("Category").agg(sum("Quantity").alias("Number of Products Sold")).orderBy(desc("Number of Products Sold")).show(1)


+--------+-----------------------+
|Category|Number of Products Sold|
+--------+-----------------------+
|Clothing|                   3516|
+--------+-----------------------+
only showing top 1 row



**3. Category of Product that is most and least Profitable**

In [103]:
category_mod = order_deets_df.groupBy("Category").agg(sum("Profit").alias("Profit Per Category")).orderBy(desc("Profit Per Category"))
print("Category of Product is most Profitable:",category_mod.first())
print("Category of Product that is least Profitable:",category_mod.tail(1))

Category of Product is most Profitable: Row(Category='Clothing', Profit Per Category=11163.0)
Category of Product that is least Profitable: [Row(Category='Furniture', Profit Per Category=2298.0)]


**3. State with maximum orders placed**

In [116]:
most_sold = orders_df.groupBy("State").count().orderBy(desc("Count")).show(1)

+--------------+-----+
|         State|count|
+--------------+-----+
|Madhya Pradesh|  101|
+--------------+-----+
only showing top 1 row



**6. Best Customer with maximum sales**

In [270]:
#order_amount = order_deets_df.groupBy("Order ID").agg(sum(col("Amount")*col("Quantity")).alias("Total_sales")).orderBy(desc("Total_sales"))
#get the total amount per order
order_amount = order_deets_df.groupBy("Order ID").agg(sum("Amount").alias("Amount"))

#join the above df and orders_df to get the customername
best_cust = orders_df.join(order_amount,on="Order ID",how="inner")
best_cust.show(20)

#get the total_sales by aggregating the amount grouped by Customername
best_cust.groupBy("CustomerName").agg(sum("Amount").alias("Total_sales"))\
        .orderBy(desc("Total_sales")).show(1)



+--------+----------+------------+-----------------+------------------+------+
|Order ID|Order Date|CustomerName|            State|              City|Amount|
+--------+----------+------------+-----------------+------------------+------+
| B-25730|22-07-2018|     Rishabh|        Rajasthan|            Jaipur|2002.0|
| B-25787|20-09-2018|       Asish|Jammu and Kashmir|           Kashmir| 965.0|
| B-25895|04-12-2018|      Sathya|          Gujarat|             Surat| 388.0|
| B-25991|03-02-2019|       Mansi|   Madhya Pradesh|            Indore| 291.0|
| B-25660|16-05-2018|      Shruti|        Karnataka|         Bangalore| 245.0|
| B-25788|21-09-2018|      Dinesh|       Tamil Nadu|           Chennai|  12.0|
| B-25693|18-06-2018|       Parna|   Madhya Pradesh|            Bhopal| 975.0|
| B-25799|01-10-2018|    Divyansh|          Gujarat|         Ahmedabad| 367.0|
| B-25812|10-10-2018|     Kshitij|   Madhya Pradesh|            Indore| 259.0|
| B-25888|02-12-2018|       Rohan|           Punjab|

Multiplying two columns using selectExpr

In [128]:
Total_amount = order_deets_df.select("Order ID","Amount", "Quantity").selectExpr("Amount * Quantity").show()

+-------------------+
|(Amount * Quantity)|
+-------------------+
|             8925.0|
|              330.0|
|               24.0|
|              320.0|
|              336.0|
|             2120.0|
|            10468.0|
|             1683.0|
|              952.0|
|             6775.0|
|               24.0|
|              579.0|
|              540.0|
|              464.0|
|              642.0|
|               24.0|
|               38.0|
|              130.0|
|             1413.0|
|              525.0|
+-------------------+
only showing top 20 rows



**5. Checking target achieved status**

In [145]:
#to check the target achieved status, we have to work with all the three datasets based on date and category 
target_df.show()

+-------------------+---------+-------+
|Month of Order Date| Category| Target|
+-------------------+---------+-------+
|             Apr-18|Furniture|10400.0|
|             May-18|Furniture|10500.0|
|             Jun-18|Furniture|10600.0|
|             Jul-18|Furniture|10800.0|
|             Aug-18|Furniture|10900.0|
|             Sep-18|Furniture|11000.0|
|             Oct-18|Furniture|11100.0|
|             Nov-18|Furniture|11300.0|
|             Dec-18|Furniture|11400.0|
|             Jan-19|Furniture|11500.0|
|             Feb-19|Furniture|11600.0|
|             Mar-19|Furniture|11800.0|
|             Apr-18| Clothing|12000.0|
|             May-18| Clothing|12000.0|
|             Jun-18| Clothing|12000.0|
|             Jul-18| Clothing|14000.0|
|             Aug-18| Clothing|14000.0|
|             Sep-18| Clothing|14000.0|
|             Oct-18| Clothing|16000.0|
|             Nov-18| Clothing|16000.0|
+-------------------+---------+-------+
only showing top 20 rows



In [146]:
#from this table we have to focus on date and order_id and modify the order date column to the required format in Month of order date column in
#target_df table.  
orders_df.show()


+--------+----------+------------+-----------------+------------------+
|Order ID|Order Date|CustomerName|            State|              City|
+--------+----------+------------+-----------------+------------------+
| B-25601|01-04-2018|      Bharat|          Gujarat|         Ahmedabad|
| B-25602|01-04-2018|       Pearl|      Maharashtra|              Pune|
| B-25603|03-04-2018|       Jahan|   Madhya Pradesh|            Bhopal|
| B-25604|03-04-2018|      Divsha|        Rajasthan|            Jaipur|
| B-25605|05-04-2018|     Kasheen|      West Bengal|           Kolkata|
| B-25606|06-04-2018|       Hazel|        Karnataka|         Bangalore|
| B-25607|06-04-2018|    Sonakshi|Jammu and Kashmir|           Kashmir|
| B-25608|08-04-2018|     Aarushi|       Tamil Nadu|           Chennai|
| B-25609|09-04-2018|      Jitesh|    Uttar Pradesh|           Lucknow|
| B-25610|09-04-2018|      Yogesh|            Bihar|             Patna|
| B-25611|11-04-2018|       Anita|          Kerala |Thiruvananth

In [255]:
#segregating the month and year from order_df table 
#using date_format to get the respective format required 
#Note: before using the date_format convert the string type of date to timestamp type using the to_timestamp() or cast("timestamp")

date_div = orders_df.select("Order ID","Order Date").withColumn("month",date_format(to_timestamp("Order Date","dd-MM-yyyy"),"MMM"))\
                  .withColumn("year",date_format(to_timestamp("Order Date","dd-MM-yyyy"),"yy"))

#accumulating the required columns 
order_sales = order_deets_df.join(date_div,on="Order ID",how="left")
            

In [256]:
#concatinating the month and year column by a delimiter '-' and name it as "Date"
date_table = order_sales.select(concat_ws("-","month","year").alias("Date"),"Category","Amount","Quantity")

In [271]:
#aggregate the amount grouping Date and Category column
Amount_per_cat = date_table.groupBy(["Date","Category"])\
      .agg(sum("Amount").alias("Total_Amount"))\
      .orderBy(["Total_Amount"],descending=True)
Amount_per_cat.show(10)

+------+-----------+------------+
|  Date|   Category|Total_Amount|
+------+-----------+------------+
|Jul-18|   Clothing|      2981.0|
|Jul-18|  Furniture|      3483.0|
|Jun-18|  Furniture|      5532.0|
|May-18|  Furniture|      6220.0|
|Jul-18|Electronics|      6502.0|
|Oct-18|  Furniture|      6766.0|
|Sep-18|Electronics|      7207.0|
|Apr-18|  Furniture|      8121.0|
|Sep-18|  Furniture|      8704.0|
|Jun-18|   Clothing|      8782.0|
+------+-----------+------------+
only showing top 10 rows



In [272]:
#this function is to get the status by comparing the total amount and target 
get_achieved = udf(lambda x,y: "NO" if x<y else "YES")

#to get the target and total amount in one dataframe I used join and passed the arguments to the user defined function for getting the status 
target_temp = target_df.select(col("Month of Order Date").alias("Date"),"Category","Target")
target_temp.show(10)
target = target_temp.join(Amount_per_cat,on=["Date", "Category"],how="inner")\
  .withColumn("Achieved_status",get_achieved("Total_Amount","Target")).show()

+------+---------+-------+
|  Date| Category| Target|
+------+---------+-------+
|Apr-18|Furniture|10400.0|
|May-18|Furniture|10500.0|
|Jun-18|Furniture|10600.0|
|Jul-18|Furniture|10800.0|
|Aug-18|Furniture|10900.0|
|Sep-18|Furniture|11000.0|
|Oct-18|Furniture|11100.0|
|Nov-18|Furniture|11300.0|
|Dec-18|Furniture|11400.0|
|Jan-19|Furniture|11500.0|
+------+---------+-------+
only showing top 10 rows

+------+-----------+-------+------------+---------------+
|  Date|   Category| Target|Total_Amount|Achieved_status|
+------+-----------+-------+------------+---------------+
|Feb-19|Electronics|16000.0|     12593.0|             NO|
|Nov-18|Electronics| 9000.0|     16651.0|            YES|
|Mar-19|  Furniture|11800.0|     16659.0|            YES|
|Sep-18|Electronics| 9000.0|      7207.0|             NO|
|Aug-18|   Clothing|14000.0|     11822.0|             NO|
|Apr-18|  Furniture|10400.0|      8121.0|             NO|
|Dec-18|Electronics| 9000.0|     18560.0|            YES|
|Jun-18|Electron